Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,Std Dev
0,ROAD,Construction Partners Inc,US Stocks:Industrials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EHC,Encompass Health,US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RF,Regions Financial,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VMC,Vulcan Materials Company,US Stocks:Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADTN,ADTRAN,US Stocks:Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRA,ProAssurance Corporation,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TBRG,"TruBridge, Inc.",US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MPW,"Medical Properties Trust, Inc.",US Stocks:Real Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LAKE,"Lakeland Industries, Inc.",US Stocks:Consumer Staples,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

TBRG EHC ADTN VMC PRA ROAD LAKE RF MPW 


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,TBRG,EHC,ADTN,VMC,PRA,ROAD,LAKE,RF,MPW
Date,,,,,,,,,
2019-01-02,25.00,44.92,9.58,92.51,36.80,9.56,10.42,10.72,9.94
2019-01-03,24.87,44.12,9.61,91.56,36.43,9.66,10.32,10.70,10.25
2019-01-04,25.42,45.20,9.94,96.25,37.99,9.55,10.65,11.10,10.21
2019-01-07,25.81,46.33,10.19,98.16,37.84,9.75,10.91,11.29,10.38
2019-01-08,26.18,47.31,10.45,97.77,37.69,10.49,11.25,11.40,10.57
...,...,...,...,...,...,...,...,...,...
2025-02-24,28.35,98.88,10.64,251.56,14.11,70.19,23.50,23.44,4.85
2025-02-25,28.91,100.63,10.43,249.79,16.13,69.60,23.05,23.37,4.81
2025-02-26,29.30,99.22,10.61,250.53,15.65,73.01,23.45,23.38,4.77


In [9]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

TBRG   209.37
EHC     33.89
ADTN    69.39
VMC     -5.50
PRA     19.85
ROAD    56.31
LAKE    27.27
RF      33.54
MPW     55.57
dtype: float64

In [10]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [11]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [12]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [13]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [14]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [15]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [16]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')